In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
# ls 

In [3]:
# mkdir FGSM-gtsrb/results

In [4]:
# !unzip  GTSRB.zip 

In [2]:
cd /gdrive/My Drive/ClassificationImages/Shape/Code

/gdrive/My Drive/ClassificationImages/Shape/Code


In [3]:
pip install torchattacks

In [4]:
from lib import *
from config import *
from model import build_model, build_model_gtsrb
from utils import * 
import torchattacks
from torchattacks import PGD, FGSM
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from os.path import isfile, join, abspath, exists, isdir, expanduser
from os import listdir
import torch.nn as nn
from torchvision import transforms, datasets, models


import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

NUM_EPOCHS = 30
BATCH_SIZE = 100


train_phase = True

attack_type = 'FGSM'

net_type = 'rgb'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:
fo = open(f'./{attack_type}-gtsrb/results/results_{net_type}.txt', 'w+')

# --------------------------------------------------------------------------------------------------------------------------------------------
# Train a model first
save_path = f'gtsrb_{net_type}.pth'


if train_phase:
    # pass
    net, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, '.')
    net.to(device)
    train_model(net, dataloader_dict, criterior, optimizer, NUM_EPOCHS, save_path)


FileNotFoundError: ignored

In [ ]:
# NUM_EPOCHS = 30 # for adversarial training

# --------------------------------------------------------------------------------------------------------------------------------------------
# Test the clean model on clean and attacks
net, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
load_model(net, save_path)
net.to(device)

acc, images = test_model_clean(net, dataloader_dict)
print('Accuracy of original model on clean images: %f ' % acc)
fo.write('Accuracy of original model on clean images: %f \n' % acc)




for eps_t in [8,32]:

    print(f'eps_t={eps_t}')
    fo.write(f'eps_t={eps_t} \n')

    epsilons = [eps_t/255]


    # Test the clean model on clean and attacks
    net, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net, save_path)
    net.to(device)

    acc_attack, images = test_model_attack(net, dataloader_dict, epsilons, attack_type, net_type, redetect_edge=False)
    print('Accuracy of clean model on adversarial images: %f %%' % acc_attack[0])
    fo.write('Accuracy of clean model on adversarial images: %f \n' % acc_attack[0])


    net, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net, save_path)
    net.to(device)

    if net_type == 'rgbedge':
        acc_attack, images = test_model_attack(net, dataloader_dict, epsilons, attack_type, net_type, redetect_edge=True)
        print('Accuracy of clean model on adversarial images with redetect_edge: %f %%' % acc_attack[0])
        fo.write('Accuracy of clean model on adversarial images with redetect_edge: %f \n' % acc_attack[0])


    # --------------------------------------------------------------------------------------------------------------------------------------------
    # Now perform adversarial training
    save_path_robust = f'./{attack_type}-gtsrb/gtsrb_{net_type}_{eps_t}_robust_{eps_t}.pth'

    if train_phase:
        # pass    
        net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
        net_robust.to(device)
        train_robust_model(net_robust, dataloader_dict, criterior, optimizer, NUM_EPOCHS, save_path_robust, attack_type, eps=eps_t/255, net_type=net_type, redetect_edge=False)


    # --------------------------------------------------------------------------------------------------------------------------------------------
    # Test the robust model on clean and attacks
    net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net_robust, save_path_robust) 
    net_robust.to(device)

    acc, images = test_model_clean(net_robust, dataloader_dict)
    print('Accuracy of robust model on clean images: %f %%' % acc)
    fo.write('Accuracy of robust model on clean images: %f \n' % acc)

    net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net_robust, save_path_robust)
    net_robust.to(device)

    acc_attack, images = test_model_attack(net_robust, dataloader_dict, epsilons, attack_type, net_type, redetect_edge=False)
    print('Accuracy of robust model on adversarial images: %f %%' % acc_attack[0])
    fo.write('Accuracy of robust model on adversarial images: %f \n' % acc_attack[0])


    net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net_robust, save_path_robust) 
    net_robust.to(device)
    
    if net_type == 'rgbedge':    
        acc_attack, images = test_model_attack(net_robust, dataloader_dict, epsilons, attack_type, net_type, redetect_edge=True)
        print('Accuracy of robust  model on adversarial images with redetect_edge: %f %%' % acc_attack[0])
        fo.write('Accuracy of robust  model on adversarial images with redetect_edge: %f \n' % acc_attack[0])




    # --------------------------------------------------------------------------------------------------------------------------------------------
    # Now perform adversarial training with redetect

    if net_type != 'rgbedge': continue

    save_path_robust = f'./{attack_type}-gtsrb/gtsrb_{net_type}_{eps_t}_robust_{eps_t}_redetect.pth'

    if train_phase:
        net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
        net_robust.to(device)
        train_robust_model(net_robust, dataloader_dict, criterior, optimizer, NUM_EPOCHS, save_path_robust, attack_type, eps=eps_t/255, net_type=net_type, redetect_edge=True)

    net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net_robust, save_path_robust) 
    net_robust.to(device)

    acc, images = test_model_clean(net_robust, dataloader_dict)
    print('Accuracy of robust redetect model on clean images: %f %%' % acc)
    fo.write('Accuracy of robust redetect model on clean images: %f \n' % acc)

    net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net_robust, save_path_robust) 
    net_robust.to(device)

    acc_attack, images = test_model_attack(net_robust, dataloader_dict, epsilons, attack_type, net_type, redetect_edge=False)
    print('Accuracy of robust redetect  model on adversarial images: %f %%' % acc_attack[0])
    fo.write('Accuracy of robust redetect  model on adversarial images: %f \n' % acc_attack[0])


    net_robust, dataloader_dict, criterior, optimizer = build_model_gtsrb(net_type, './gtsrb/')
    load_model(net_robust, save_path_robust)
    net_robust.to(device)
    
    acc_attack, images = test_model_attack(net_robust, dataloader_dict, epsilons, attack_type, net_type, redetect_edge=True)
    print('Accuracy of robust redtect model on adversarial images with redetect_edge: %f %%' % acc_attack[0])
    fo.write('Accuracy of robust redetect model on adversarial images with redetect_edge: %f \n' % acc_attack[0])




fo.close()



Model loaded ...
device: cuda:0
Accuracy of original model on clean images: 0.452494 
eps_t=8
Model loaded ...
device: cuda:0
Accuracy of clean model on adversarial images: 0.178385 %
Model loaded ...
device: cuda:0
Accuracy of clean model on adversarial images with redetect_edge: 0.127553 %
Model loaded ...
device: cuda:0
Epoch 1/30
Epoch 2/30
train Loss: 3.1107, Acc: 0.2337, Acc_adv: 0.1704
Epoch 3/30
train Loss: 2.6020, Acc: 0.3501, Acc_adv: 0.2482
Epoch 4/30
train Loss: 2.4048, Acc: 0.3947, Acc_adv: 0.2810
Epoch 5/30
train Loss: 2.2806, Acc: 0.4242, Acc_adv: 0.3050
Epoch 6/30
train Loss: 2.1895, Acc: 0.4461, Acc_adv: 0.3235
Epoch 7/30
train Loss: 2.1236, Acc: 0.4643, Acc_adv: 0.3352
Epoch 8/30
train Loss: 2.0635, Acc: 0.4805, Acc_adv: 0.3497
Epoch 9/30
train Loss: 2.0127, Acc: 0.4943, Acc_adv: 0.3600
Epoch 10/30
train Loss: 1.9710, Acc: 0.5039, Acc_adv: 0.3693
Epoch 11/30
train Loss: 1.9336, Acc: 0.5121, Acc_adv: 0.3775
Epoch 12/30
train Loss: 1.8992, Acc: 0.5209, Acc_adv: 0.3860
E